<a href="https://colab.research.google.com/github/chaos44/food_recognition/blob/master/fit_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)#2. Get the file
d10 = drive.CreateFile({'id':'1jqXr5d-UAhOUaPIb-g7p_8xEW9XxHjUr'}) # replace the id with id of file you want to access
d20 = drive.CreateFile({'id':'1hbZ19igWGti65MXOeGOwuzbSzVAOLdlH'})
d30 = drive.CreateFile({'id':'12uU6ocY9r6BfBk8fo4BQC8BVXjDDhCBb'})
d40 = drive.CreateFile({'id':'1Hs5OVtYD8xDvLd7y6drDHUu4ErXjBvr_'})
d50 = drive.CreateFile({'id':'19FaeD7fdFlqu0U8K2DIJevtp1tb4PbBe'})
d10.GetContentFile('d10.zip')
d20.GetContentFile('d20.zip')
d30.GetContentFile('d30.zip')
d40.GetContentFile('d40.zip')
d50.GetContentFile('d50.zip')

In [ ]:
!unzip d10.zip -d food
!unzip d20.zip -d food
!unzip d30.zip -d food
!unzip d40.zip -d food
!unzip d50.zip -d food

In [3]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# 中心から任意のサイズをトリミング
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [6]:
# パラメータの初期化
classes = ['beefdon', 'curry', 'fish', 'humberger', 'katsudon', 'ramen', 'seafooddon', 'soba', 'spaghetti', 'tempuradon'
         , 'udon', 'lasagna', 'hamburger_steak', 'nikujaga', 'fried_rice', 'omelette', 'sukiyaki', 'sushi', 'gyoza', 'fried_chicken'
         , 'okonomiyaki', 'croquette', 'pancake', 'french_toast', 'tart', 'taco_rice', 'doria', 'takoyaki', 'tapioca', 'yakisoba'
         , 'ginger_pork', 'steak', 'roast_meat', 'bread', 'yakitori', 'rice_ball', 'goya_chanpuru', 'instant_noodle', 'meat_bun', 'gratin'
         , 'mabo_tofu', 'tiramisu', 'cream_puff', 'eclair', 'hot_dog', 'potato_salad', 'stew', 'pizza', 'french_fry', 'avocado_salad']
num_classes = len(classes)
image_size = 150

In [8]:
# 画像の読み込みとnumpy配列への変換
X_test = [] # リスト
y_test = [] # リスト

In [9]:
# アスペクト比を固定して、幅が指定した値になるようリサイズする。
def scale_to_width(img, width):
    height = round(img.height * width / img.width)
    return img.resize((width, height))

In [10]:
def generate(classes):
    for index, classlabel in enumerate(classes):
        photos_dir = './food/' + classlabel  
        files = glob.glob(photos_dir + '/*.jpg')
        count = len(files)
        # print(classlabel, count)  
        for i, file in enumerate(files):
            image = Image.open(file)
            image = image.convert('RGB')
            # image = image.convert('RGB')の前にトリミングするとメモリオーバー、理由不明
            # 食材ごとにテストとトレーニングに分ける
            # 画像サイズの取得
            # 長い方に沿ってトリミング
            # image = crop_center(image, 400, 400) # resizeする時にアスペクト比を固定させるため、400*400の部分をトリミング(800 * 800より正解率が上がる)
            w, h = image.size
            image = crop_center(image, w, w) if w >= h else crop_center(image, h, h)
            image = image.resize((image_size, image_size))
            data = np.asarray(image, dtype=np.float32) / 255.0  # 正規化 # dtype=np.float32でサイズ縮小、MemoryError解消のため # 2020/7/7
            yield (data, index)
        if (i > 3/4 * count):
            X_test.append(data)
            y_test.append(index)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
for index, classlabel in enumerate(classes):
    photos_dir = './food/' + classlabel  
    files = glob.glob(photos_dir + '/*.jpg')
    count = len(files)
    # print(classlabel, count)  
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert('RGB')
        # image = image.convert('RGB')の前にトリミングするとメモリオーバー、理由不明
        # 食材ごとにテストとトレーニングに分ける
        # 画像サイズの取得
        # 長い方に沿ってトリミング
        # image = crop_center(image, 400, 400) # resizeする時にアスペクト比を固定させるため、400*400の部分をトリミング(800 * 800より正解率が上がる)
        w, h = image.size
        image = crop_center(image, w, w) if w >= h else crop_center(image, h, h)
        image = image.resize((image_size, image_size))
        data = np.asarray(image, dtype=np.float32) / 255.0  # 正規化 # dtype=np.float32でサイズ縮小、MemoryError解消のため # 2020/7/7
        if (i < 3/4 * count):
            X_train.append(data)
            y_train.append(index)
        else: 
            X_test.append(data)
            y_test.append(index)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [12]:
print(X_test.nbytes)
print(y_test.nbytes)

0
0


In [ ]:
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y)
# X_train, X_test = model_selection.train_test_split(X)
# print(X_train.nbytes)
# print(X_test.nbytes)

In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils import np_utils
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications import EfficientNetB4

y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [14]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [15]:
# compute quantities requiredXXfor featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

ValueError: ignored

In [16]:
# mobile_net
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:])) # 1番目は個数、その後の数を使う
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))

model = Model(inputs=model.input, outputs=top_model(model.output))

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
# レイアを調べる
for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [19]:
# 最後のブロックだけ学習させる
# for layer in model.layers[0:154]:
#    layer.trainable = False

# block_16学習
for layer in model.layers[0:143]:
    layer.trainable = False
opt = Adam(lr=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(generate(classes), steps_per_epoch=32, epochs=150)

# score = model.evaluate(X_test, y_test, batch_size=32)
# print(score)

AttributeError: ignored

In [ ]:
model.save('./food_recognition_mobile.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def process(data):
    plt.figure()
    plt.imshow(data)

In [ ]:
print(X_test.shape)

(4210, 150, 150, 3)


In [ ]:
# 比較用 800*800 トリミング
for index, classlabel in enumerate(classes):
    photos_dir = './food/' + classlabel  
    files = glob.glob(photos_dir + '/*.jpg')
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert('RGB')
        # image = image.convert('RGB')の前にトリミングするとメモリオーバー、理由不明
        image = crop_center(image, 800, 800) # resizeする時にアスペクト比を固定させるため、800*800の部分をトリミング
        image = image.resize((image_size, image_size))
        data = np.asarray(image, dtype=np.float32) / 255.0  # 正規化 # dtype=np.float32でサイズ縮小、MemoryError解消のため # 2020/7/7
        data = np.array(data)
        process(data)
        X = []
        # 最後尾に追加する
        X.append(data)
        X = np.array(X)
        result = model.predict([X])[0]
        # 値の大きい方の番号(配列の添字)を返す
        predicted = result.argmax()
        percentage = int(result[predicted] * 100)
        print(classes[predicted], percentage)
        print()
        if i > 0:
          break

In [ ]:
for index, classlabel in enumerate(classes):
    photos_dir = './food/' + classlabel  
    files = glob.glob(photos_dir + '/*.jpg')
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert('RGB')
        w, h = image.size
        image = crop_center(image, w, w) if w >= h else crop_center(image, h, h)
        image = image.resize((image_size, image_size))
        data = np.asarray(image, dtype=np.float32) / 255.0  # 正規化 # dtype=np.float32でサイズ縮小、MemoryError解消のため # 2020/7/7
        data = np.array(data)
        process(data)
        X = []
        # 最後尾に追加する
        X.append(data)
        X = np.array(X)
        result = model.predict([X])[0]
        # 値の大きい方の番号(配列の添字)を返す
        predicted = result.argmax()
        percentage = int(result[predicted] * 100)
        print(classes[predicted], percentage)
        print()
        if i > 0:
          break